In [ ]:
import itertools
import os
import matplotlib.pylab as plt
import numpy as np
import tensorflow as tf
import tensorflow_hub as hub
# import math

#checking framework versions
from tensorflow.python.client import device_lib

#for checking hardware
import subprocess
import pprint

#mounting gDrive
from google.colab import drive

#preprocessing

from keras.preprocessing.image import ImageDataGenerator

#callbacks
from keras.callbacks import ReduceLROnPlateau, ModelCheckpoint, EarlyStopping

#Confusion Matrix
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.metrics import ConfusionMatrixDisplay


#Andreea Stuff
from tensorflow import keras
from keras import layers
from keras.layers.core import Dense, Activation, Flatten, Dropout
from keras.optimizers import Adam
from keras.metrics import categorical_crossentropy, Accuracy
from keras.models import Model
from keras.models import save_model, load_model
from keras.layers import Dense,GlobalAveragePooling2D

Version checking + Hardware checking

In [ ]:
print("TF version:", tf.__version__)
print("Hub version:", hub.__version__)
print("GPU is", "available" if tf.config.list_physical_devices('GPU') else "NOT AVAILABLE")
tf.test.gpu_device_name()
device_lib.list_local_devices()

TF version: 2.4.1
Hub version: 0.11.0
GPU is NOT AVAILABLE


[name: "/device:CPU:0"
 device_type: "CPU"
 memory_limit: 268435456
 locality {
 }
 incarnation: 14543054048938878537]

In [ ]:
sp = subprocess.Popen(['nvidia-smi', '-q'], stdout=subprocess.PIPE, stderr=subprocess.PIPE)

out_str = sp.communicate()
out_list = str(out_str[0]).split('\\n')

out_dict = {}

for item in out_list:
    print(item)

In [ ]:
!cat /proc/meminfo

## Select the TF2 SavedModel

Here we select our  TF2 SavedModel

In [ ]:
module_selection = ("inception_v3", 299) #@param ["(\"mobilenet_v2_100_224\", 224)", "(\"resnet_v2_50\", 224)","(\"inception_v3\", 299)"] {type:"raw", allow-input: true}
handle_base, pixels = module_selection

MODULE_HANDLE ="https://tfhub.dev/google/imagenet/{}/feature_vector/4".format(handle_base)
IMAGE_SIZE = (pixels, pixels)
print("Using {} with input size {}".format(MODULE_HANDLE, IMAGE_SIZE))

# Names of the integer classes, i.e., 0 -> afraid, 1 -> angry, etc.
class_names = ['afraid', 'angry', 'disgusted', 'happy', 'neutral', 'sad', 'surprised']

BATCH_SIZE = 32

#Connecting to gDrive

In [ ]:
drive.mount('/content/gdrive')

In [ ]:
train_dir = "/content/gdrive/MyDrive/EmotionRec/Affectnet" 
test_val_dir = "/content/gdrive/MyDrive/EmotionRec/KDEF"

#Preprocessing

Dataset was modified in Greyscaled and cropped using Haarcascade. This was done in Comand Prompt because it shrinked the dataset by 1/3 and it proved a faster training. (30 min faster on the first batch). This also helped us in transfering the dataset due to cloud transfer constrains.

#Setting up the dataset

In [ ]:
preprocess_function = tf.keras.applications.inception_v3.preprocess_input #@param ["tf.keras.applications.inception_v3.preprocess_input","tf.keras.applications.resnet.preprocess_input", "tf.keras.applications.mobilenet.preprocess_input"] {type:"raw", allow-input: true}
train_batches = ImageDataGenerator(preprocessing_function= preprocess_function).flow_from_directory(directory=train_dir,target_size=(IMG_HEIGHT,IMG_WIDTH),classes=list(CLASS_NAMES),batch_size=BATCH_SIZE)
valid_batches = ImageDataGenerator(preprocessing_function= preprocess_function).flow_from_directory(directory=test_val_dir,target_size=(IMG_HEIGHT,IMG_WIDTH),classes=list(CLASS_NAMES),batch_size=BATCH_SIZE)

In [ ]:
train_img_batch,train_label_batch = next(train_batches) 
test_img_batch,test_label_batch=next(valid_batches)   #Iterating over the Generator

In [ ]:
train_label_batch,test_label_batch=train_label_batch.astype('uint32'),test_label_batch.astype('uint32') 

In [ ]:
train_ds=tf.data.Dataset.from_tensor_slices((train_img_batch,train_label_batch))
test_ds=tf.data.Dataset.from_tensor_slices((test_img_batch,test_label_batch))

In [ ]:
train_ds = train_ds.shuffle(1024).repeat().batch(BATCH_SIZE).cache()
test_ds = test_ds.repeat().batch(BATCH_SIZE).cache()

# Defining the model

All it takes is to put a linear classifier on top of the `feature_extractor_layer` with the Hub module.

For speed, we start out with a non-trainable `feature_extractor_layer`, but you can also enable fine-tuning for greater accuracy.

First Phase Fine Tuning


In [ ]:
# # # FINE - TUNING 
# base_model.trainable = True

# # Let's take a look to see how many layers are in the base model
# print("Number of layers in the base model: ", len(base_model.layers))

# # Fine-tune from this layer onwards
# fine_tune_at = 100

# # Freeze all the layers before the `fine_tune_at` layer
# for layer in base_model.layers[:fine_tune_at]:
#   layer.trainable =  False


# model.compile(optimizer=keras.optimizers.Adam(learning_rate=0.00001),  # Optimizer
#   # Loss function to minimize
#   loss=keras.losses.CategoricalCrossentropy(),
#   # List of metrics to monitor
#   metrics=[keras.metrics.Accuracy()])

In [ ]:
do_fine_tuning = True #@param {type:"boolean"}


*   Regularisation - a process of introducing additional information in order to prevent overfitting. 

> L2 generally beats L1 in terms of accuracy and it is easier to adjust.

> loss = l2 * reduce_sum(square(x))



 

In [ ]:
print("Building model with", MODULE_HANDLE)

model = tf.keras.Sequential([
    # Explicitly define the input shape so the model can be properly
    # loaded by the TFLiteConverter
    tf.keras.layers.InputLayer(input_shape=IMAGE_SIZE + (3,)),
    hub.KerasLayer(MODULE_HANDLE, trainable=do_fine_tuning),
    tf.keras.layers.Dropout(rate=0.2),
    tf.keras.layers.Dense(len(class_names),activation='softmax',
                          kernel_regularizer=tf.keras.regularizers.l2(0.0001))
])
model.build((None,)+IMAGE_SIZE+(3,)) #groups layers into an object with training and inference features.
model.summary()

Used for MobileNet

In [ ]:
# # functional method of adding layers

# inputs = tf.keras.Input(shape=(img_width, img_height, 3))
# x = data_augmentation(inputs)
# x = resize_and_rescale(x)

# x=base_model(x)

# # x=base_model.output
# x=GlobalAveragePooling2D()(x)
# x=Flatten()(x) #connection between cov layers and dense layers

# x=Dense(512,activation='relu')(x) 
# x=Dropout(0.5)(x)

# x=Dense(512*2,activation='relu')(x) #we add dense layers so that the model can learn more complex functions and classify for better results.
# x=Dropout(0.5)(x) #setting parameters to 0 which helps prevent overfitting

# x=Dense(512*4,activation='relu')(x) #dense layer 2
# x=Dropout(0.5)(x)
# x=layers.BatchNormalization()(x)

# preds=Dense(7,activation='softmax')(x) #final layer with softmax activation

# model=Model(inputs=inputs,outputs=preds)

# Training the model

*   Stochastic gradient descent Optimizer

  *   Optimizer helps us to minimize an error function(loss function)or to maximize the efficiency of production.
  *   Momentum is like a ball rolling downhill. The ball will gain momentum as it rolls down the hill.
*  Observation:
  *   SGD without momentum , Updates takes longer vertical steps [slower learning]than horizontal step[faster learning].
  *   SGD with momentum,Updates takes longer Horizontal steps[faster learning] then vertical step[slower learning].







*   SparseCategoricalCrossentropy is more efficient when you have a lot of categories.



In [ ]:
model.compile(
  optimizer=tf.keras.optimizers.SGD(lr=0.005, momentum=0.9), 
  loss=keras.losses.SparseCategoricalCrossentropy(),
  metrics=['accuracy'])

Used for the 1st Phase

In [ ]:
# model.compile(optimizer=keras.optimizers.Adam(learning_rate=1e-4),  # Optimizer close to 0.0075
#   # Loss function to minimize
#   loss=keras.losses.SparseCategoricalCrossentropy(), # vs CategoricalCrossentropy() because our data is aranges so that for each emotion we got a value, w
#   # List of metrics to monitor
#   metrics=[keras.metrics.SparseCategoricalAccuracy()])

#Checkpoints

ReduceLROnPlateau - Reduce learning rate when a metric has stopped improving.

> Models often benefit from reducing the learning rate by a factor of 2-10 once learning stagnates. This callback monitors a quantity and if no improvement is seen for a 'patience' number of epochs, the learning rate is reduced.






In [ ]:
# CHECKPOINTS USED

# early checkpoint. It stops when the validation dataset starts to degrade
early = EarlyStopping(monitor='val_loss',patience=10,mode='auto')

# create a model checkpoint
model_checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
        filepath="/content/drive/MyDriveadded_dense_weights.hdf5", # Path where to save the WEIGHTS
        save_best_only=True,  # Only save a model if `val_loss` has improved.
        monitor="val_loss",
        mode='min', # i want the min value of the loss
        verbose=1)

# create a ReduceLROnPlateau callback

reduce_lr = ReduceLROnPlateau(
    monitor='val_loss', 
    factor=0.1,   # reduce the rate with 0.1
    patience=1, 
    min_lr=0.0001,
    verbose=1
)
callbacks_list = [early,model_checkpoint_callback,reduce_lr]

In [ ]:
# #Automatic rename with epoch number and val accuracy:
# from keras.callbacks import *
# filepath="/content/gdrive/MyDrive/MyCNN/epochs:{epoch:03d}-val_accuracy:{val_accuracy:.3f}.hdf5"
# checkpoint = ModelCheckpoint(filepath, monitor='val_accuracy', verbose=1, save_best_only=True, mode='max')
# callbacks_list = [checkpoint]

#Training



*   Epochs-One Epoch is when an ENTIRE dataset is passed forward and backward through the neural network only ONCE.
*   Batch Size - Total number of training examples present in a single batch.
*   Batch - Defines the number of samples that will be propagated through the network.
*   Iterations - The number of batches needed to complete one epoch.






In [ ]:
STEPS_PER_EPOCH=np.ceil(len(train_batches.filenames)/BATCH_SIZE)
VALIDATION_STEPS=np.ceil(len(valid_batches.filenames)/BATCH_SIZE)
print(STEPS_PER_EPOCH , VALIDATION_STEPS)

In [ ]:
# Train the classifier.
hist = model.fit(
    train_ds,
    epochs=100, 
    steps_per_epoch=STEPS_PER_EPOCH ,
    callbacks=callbacks_list,
    validation_data=valid_ds,
    validation_steps=VALIDATION_STEPS).history

In [ ]:
plt.figure()
plt.ylabel("Loss (training and validation)")
plt.xlabel("Training Steps")
plt.ylim([0,2])
plt.plot(hist["loss"])
plt.plot(hist["val_loss"])

plt.figure()
plt.ylabel("Accuracy (training and validation)")
plt.xlabel("Training Steps")
plt.ylim([0,1])
plt.plot(hist["accuracy"])
plt.plot(hist["val_accuracy"])

#Confusion Matrix

In [ ]:
# COMPUTE CONFUSION MATRIX NORMALIZED

Y_pred = model.predict(valid_generator, valid_generator.samples // BATCH_SIZE+1)
y_pred = np.argmax(Y_pred, axis=1)

print('Confusion Matrix')
cm = confusion_matrix(valid_generator.classes, y_pred)

# normalize the data
cm = cm / cm.astype(np.float).sum(axis=1)[:, np.newaxis] # recall  (tp)/(tp+fn) 

print(cm)
print('Classification Report')
target_names = list(train_generator.class_indices.keys())
print(classification_report(valid_generator.classes, y_pred, target_names=target_names))


disp = ConfusionMatrixDisplay(confusion_matrix=cm,
                              display_labels=valid_generator.classes)

# make the plot bigger
fig, ax = plt.subplots(figsize=(10, 10))
disp.plot(cmap=plt.cm.Blues, ax=ax)

Finally, the trained model can be saved for deployment to TF Serving or TF Lite (on mobile) as follows.

In [ ]:
model_name = "Inception-v3.h5"#@param["Inception-v3.h5","ResNetV2.h5","MobileNet.h5"]
model.save(model_name)
model.save("/content/gdrive/MyDrive/MyCNN/"+model_name)

#Data Augmentation

In [ ]:
# datagen_kwargs = dict(rescale=1./255, validation_split=.20)
# dataflow_kwargs = dict(color_mode="rgb",target_size=IMAGE_SIZE, batch_size=BATCH_SIZE,
#                    interpolation="bilinear")

# valid_datagen = tf.keras.preprocessing.image.ImageDataGenerator(
#     **datagen_kwargs)
# valid_generator = valid_datagen.flow_from_directory(
#     test_val_dir, subset="validation", shuffle=False, **dataflow_kwargs) #test_val_dir

# do_data_augmentation = True #@param {type:"boolean"}
# if do_data_augmentation:
#   train_datagen = tf.keras.preprocessing.image.ImageDataGenerator(
#       rotation_range=40,
#       horizontal_flip=True,
#       width_shift_range=0.2, 
#       height_shift_range=0.2,
#       shear_range=0.2, zoom_range=0.2,
#       **datagen_kwargs)
# else:
#   train_datagen = valid_datagen
# train_generator = train_datagen.flow_from_directory(
#     train_dir, subset="training", shuffle=True, **dataflow_kwargs) #train_dir


# number_of_valid_examples = len(valid_generator.filenames)
# number_of_generator_calls = math.ceil(number_of_valid_examples / (1.0 * BATCH_SIZE)) 
# # 1.0 above is to skip integer division

# valid_labels = []

# for i in range(0,int(number_of_generator_calls)):
#     valid_labels.extend(np.array(valid_generator[i][1]))